In [1]:
import sys
sys.path.append('../')
from SerbSynpretproceing import showGrid
from SerbainTagger import SrbTreeTagger
from srpskiwordnet import SrbWordNetReader
from srppolsets import PolaritySets, SrbSynset2GlossTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from StemmerByNikola import stem_str
import numpy as np
from sklearn.neural_network import MLPClassifier
from srpskiwordnet import swordnet
def load_file_into_list(filename):
    with open(filename, mode="r", encoding="utf-16") as file:
        lines = [line.strip() for line in file]
    return lines
from sklearn.naive_bayes import ComplementNB, BernoulliNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import AdaBoostClassifier


C:\ProgramData\Anaconda3\envs\ML_extra\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
C:\ProgramData\Anaconda3\envs\ML_extra\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
C:\ProgramData\Anaconda3\envs\ML_extra\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
C:\ProgramData\Anaconda3\envs\ML_extra\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


ImportError: cannot import name 'swordnet' from 'srpskiwordnet' (C:\Users\Korisnik\Documents\Python\NLTK\srpskiwordnet.py)

In [ ]:
C_range = np.logspace(1, 10, 3)
gamma_range = np.logspace(-9, 3, 6)
alpha_range = np.logspace(-9, 3, 12)

# stop_words = [".", "i", "ili", "(", ")", ";", ",", "u", "iz", "se", "koji",
#               "na", "kao", "sa", "kojim", "koj"]
RES_DIR = ".\\resources\\"
stop_words1 = load_file_into_list(RES_DIR + "stopwordsSRB.txt")
stop_words2 = load_file_into_list(RES_DIR + "stopwordsSRB tfidf.txt")
stop_words3 = load_file_into_list(RES_DIR + "stopwordsSRB tf.txt")
tt = SrbTreeTagger()
stop_words4 = list()
for word in stop_words3:
    pom = tt.lemmarizer(word)
    if not pom in stop_words4:
        stop_words4.append(pom)
swordnet.parse_all_defintions(tt.lemmarizer)
Synset_Sentiment_set = PolaritySets(swordnet, 0)

In [ ]:
Synset_Sentiment_set.addWSWN()

In [ ]:
positive_seed_words = ["dobar",
                       "dobrota",
                       "lep",
                       "čudesno",
                       "dragocen",
                       "anđeoski",
                       "izobilje",
                       "izbavljenje",
                       "tešiti",
                       "ispravnost",
                       "oduševiti se",
                       "slast",
                       "uveseljavajući",
                       "napredovati",
                       "proslavljen",
                       "usrećiti",
                       "uspešnost"]

negative_seed_words = ["zao",
                       "antipatija",
                       "beda",
                       "bedan",
                       "bol",
                       "laž",
                       "lažno",
                       "korupcija",
                       "krivica",
                       "prezreti",
                       "tuga",
                       "nauditi",
                       "sebičnost",
                       "paćeništvo",
                       "ukloniti s ovog sveta",
                       "masakr",
                       "ratovanje"]
positive_seed_IDS = ["ENG30-01828736-v",
                     "ENG30-13987905-n",
                     "ENG30-01777210-v",
                     "ENG30-13987423-n"]

negative_seed_IDS = ["ENG30-01510444-a",
                     "ENG30-01327301-v",
                     "ENG30-00735936-n",
                     "ENG30-00220956-a",
                     "ENG30-02463582-a",
                     "ENG30-01230387-a",
                     "ENG30-00193480-a",
                     "ENG30-00364881-a",
                     "ENG30-14213199-n",
                     "ENG30-01792567-v",
                     "ENG30-07427060-n",
                     "ENG30-14408086-n",
                     "ENG30-14365741-n",
                     "ENG30-02466111-a"]

Synset_Sentiment_set.addPOSall(positive_seed_words)
Synset_Sentiment_set.addNEGall(negative_seed_words)
Synset_Sentiment_set.addPOSIDall(positive_seed_IDS)
Synset_Sentiment_set.addNEGIDall(negative_seed_IDS)

In [ ]:
Synset_Sentiment_set.updateDataFrame()

Synset_Sentiment_sets = list()
Synset_Sentiment_sets.append(Synset_Sentiment_set)

for _ in range(6):
    Synset_Sentiment_sets.append(
        Synset_Sentiment_sets[-1].next_itteration())

In [ ]:
pipeline = Pipeline(
    [
        ("gloss", SrbSynset2GlossTransformer()),
        ("vect", HashingVectorizer(stop_words=stop_words2)),
        ("tfidf", TfidfTransformer())]
)

In [ ]:
pipeline

In [ ]:
X, y = Synset_Sentiment_set.getXY()

In [ ]:
X = pipeline.fit_transform(X, y)

In [ ]:
X

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [ ]:

min_features_to_select = 1  # Minimum number of features to consider
clf = LogisticRegression()
cv = StratifiedKFold(10)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=-1,
)
rfecv.fit(X, y)

print(f"Optimal number of features: {rfecv.n_features_}")